### Lindsey Bodenbender
## Introduction
This retail analysis project utilizes a Zara 2025 sales activity dataset representing clothes sales. The goal of the project is to practice using Power BI on a real dataset to answer questions about factors that drive sale performance. To view the pbix file, download it and open it using Power BI desktop.

## About the dataset
The dataset is a csv file containing 17 attributes, 999+ rows, and the following notable attributes:
- `Product ID`
- `Product Position` (e.g. Aisle, End-cap..)
- `Promotion` (Yes/No)
- `Sales Volume`
- `Name`
- `Description`

A link to the dataset:
https://www.kaggle.com/datasets/tan5577/business-salesdataset2025/data

# Outline of the analysis to perform and questions to answer
### 1. Product performance and sales drivers <br>
**Goal: Understand which product characteristics most influence sales success.**
- Which products drive the highest revenue?
- Are sales more strongly influenced by season or product placement?


### 2. Regional and Seasonal Revenue Analysis <br>
**Goal: Explore how product origin and seasonal demand influence overall business performance.**
- Which countries of origin generate the highest revenue? Are certain regions consistently stronger performers for specific product types?
- Which season outperform or underperform compared to the average?
- What does the forecast indicate about next season's expected demand?


### 3. Underperforming Products <br>
**Goal: Identify how many products are selling under the average for their category and view details about what factors affect this.**
- Which products are underperforming relative to the average revenue of their product type?
- What patterns emerge between price, revenue, and underperforming products?
- Do promotions, placement, or seasonality negatively impact revenue?

# Data Cleaning/Preparation

1. **Imported and Profiled the Dataset** <br>
First, I imported the dataset into Power BI using the Get Data feature. Power BI automatically promoted the first row to the header and assigned initial data types based off the top 200 rows. I then used the Power Query Editor to profile the data.
- Using the **column quality** feature, I validated the dataset for errors or missing values. No errors were dectected, and fewer than 1% of the values were missing in the `Description` column, which required no corrective action.
- Using **column distribution** I reviewed the uniqueness and distinctness of each field. As expected, `Product ID` contains only unique values, making it suitable as the primary key. 
I also identified that `Product Category`, `Brand`, and `Currency` each only contained one distinct value. Since these columns would not contribute to meaningful analysis, I removed them to streamline the dataset.

2. **Renamed and Standardized Columns** <br>
To improve readability and ensure naming consistency across the model, I renamed several fields:
- Renamed `Name` to `Product Name` for clarity.
- Renamed `terms` to `Product Type`, as the original column name `terms` was vague and didn't accurately describe the columns purpose.

3. **Applied Appropriate Data Categories** <br>
Assigned correct data categories to ensure accurate behavior within visuals:
- Set `URL` to Web URL, enabling clickable links in the report. (Note: Ended up removing this column as it did not contribute to my analysis)
- Set `Country` to Country/Region so it can be used efficiently in map visuals.

4. **Text Formatting Enhancements** <br>
To improve the consistency and presentation of categorical fields, I applied proper casing to the `Section` column so that the column name is capitalized rather than displayed in all uppercase.

5. **Refined the `Product Type` Field** <br>
Since each row represents a single product, I standardized product type naming to a singular form.
- I used the following M formula to create a new custom column that removes the trailing "s" from each product type: <br>
= Table.AddColumn(#"Renamed Columns1", "Custom", each Text.Start([Product Type], Text.Length([Product Type]) - 1))
- After generating the corrected values, I used the replace values feature to replace instances of Shoe with Shoes to retain appropriate pluralization where needed.

# Create Star Schema Model
I started by renaming the imported dataset to Fact_Sales, which serves as the central fact table containing all transactional sales records.

### Dimension Tables
#### Dim_Product
- To build the product dimension table, I duplicated Fact_Sales, retaining only the attribute-level fields relevant for product analysis, including: `Product ID`, `Product Name`, `Description`, `Product Type`, `Section`, `Material`, `Origin`, `URL`, and `Price`.
- I verified that the `Product ID` column contains no duplicates, confirming that each product functions as a unique key.

#### Dim_Position
- I duplicated the Fact_Sales table to construct Dim_Position, keeping only the `Position Name` column and removing all others.
- Removed duplicates from the `Position Name` column

#### Dim_Season
- Since the dataset only contains seasonal information rather than full dates, I created a Dim_Season table to serve as the model's time dimension.

#### Relationships
- I configured one-to-many relationships from each dimension table to Fact_Sales, using their corresponding keys (`Product ID`, `Position Name`, and `Season`). This aligns with a standard star schema design, where each dimension table provides descriptive context for the central fact data.

![Model](./Images/Model.png)

# Analysis
### Product Performance <br>
To evaluate which products drive the strongest revenue outcomes, I created a Top 10 Products by Sales Revenue bar chart. This is powered by a custom measure: <br>
Sales Revenue = SUMX(Fact_Sales, Fact_Sales[Sales Volume] * Fact_Sales[Price])
This measure calculates revenue at the row level to ensure accurate aggregation.

![Top Products by Sales Revenue](./Images/Top%20Products%20by%20Sales%20Revenue.png)

- To enhance interactivity and enable targeted analysis, I added slicers for `Product Type`, `Season`, and `Section`. I configured the visual interaction settings so that only the Top Products chart and the total revenue card respond to these filters. This allows for focused revenue exploration without unintentionally affecting unrelated visuals.
- The decomposition tree was added to break down sales volume drivers, allowing users to analyze how factors such as product position and season contribute to performance.
- Additionally, I created a bar chart showing the average revenue by product type, enabling quick comparisons of which categories contribute the most on a per-product basis.

### Regional and Seasonal Analysis <br>
To explore geographic trends, I added a map visual summarizing total revenue by country of origin.

![Total Sales Revenue by Country of Origin](./Images/Total%20Revenue%20by%20Country%20of%20Origin.png)

Seasonal performance was analyzed using several visuals:
- A bar chart showing sales volume by season with an average line, helping identify seasons that overperform or underperform relative to the overall benchmark.
- A line chart visualizing sales patterns across seasons to highlight cyclical trends. 
- I wanted to utilize Power BI's built-in forecast model to project future sales, so I simulated seasonal cycles by assigning representative dates to seasons, providing insights into potential future seasonal sales patterns.
- Slicers for `Product Type` and `Season` were also added to allow users to refine these views.

### Underperforming Products <br>
To identify products that fall below expectations, I created a measure to classify underperformance: <br>
Is Underperforming = 
IF( [Sales Revenue] < [Average Sales], 1, 0)

From this, I developed several KPI cards displaying:
- The average price of underperforming products
- The average price of adequately performing products
- The percentage of products that are underperforming
- Seasonal vs. non-seasonal revenue (in thousands)
- Promo vs. non-promo revenue
These metrics help uncover pricing trends and highlight whether certain product characteristics correlate with lower sales.

- To futher analyze performance drivers, I added a scatter plot comparing price vs. revenue, with underperforming products visually distinguished.

- Finally, I created a matrix for viewing detailed information about products that perform below their product type average. Sorting this table in ascending order highligts the lowest performers first, making it easy to identify items that may need repositioning, promotional adjustments, or removal from the catalog.

![Underperforming Products Matrix](./Images/Underperforming%20Products%20Matrix.png)

## Challenges
Several chalenges emerged during the development of the Power BI report:
- **Limited date granularity:** <br>
The dataset didn't include acutal dates, only seasonal indicators, which restricted the ability to perform true time-series analysis across months, quarters, or years. More granular date data would have enabled richer trend analysis, year-over-year comparisons, and forecasting without simulated dates.
- **Lack of cost data for profability insights:** <br>
While sales revenue could be analyzed, the absence of product cost information prevented calculations of profit margins, gross profit, or contribution analysis.
- **Performance issues with complex visuals:** <br>
Some visuals were slow to load, likely due to unnecessary columns or higher model complexity. Removing unused fields such as `Material`, `URL`, and `Description` improved performance and reduced the overall model size.
- **Determining the most valuable analytical approach:** <br>
Identifying which visuals would provide genuinely useful insights required stepping into the mindset of a sales executive. I had to consider which KPIs matter most at a srategic level, such as top-performing products, seasonal trends, and regional performance, and ensure the dashboard focused on answering those core business questions.
- **Balancing detail with usability:** <br>
Another challenge was ensuring that the report included sufficient depth (e.g., decomposition trees, underperformance segmentation) without overwhelming the user. Achieving a balance between detailed diagnostic views and high-level summaries required iterative refinement.

## Takeaways
The process of designing and refining the model and report led to several key takeaways:
- **Reusable measures are essential:** <br>
Creating clear, meaningful DAX measures early in the process enables consistency across visuals and prevents redundancies.
- **Organization improves efficiency:** <br>
Structuring measures, calculations, and tables into logical folders (and using naming conventions) made navigation easier and streamlined the development workflow.
- **User-centered thinking leads to better insights:** <br>
Designing with the end user in mind and focusing on what sales leaders actually need to understand helped prioritize visuals that offer real business value.
- **Iterative refinement is a natural part of BI development:** <br>
Many visuals were created, tested, adjusted, or replaced. This iterative process is essential to surface the cleanest and most insightful representation of the data.

## Future Work
I identified several opportunities for future development to continue refining and enhancing the report:
- **User feedback and usability testing:** <br>
I shared the current version of the report with friends and family to gather inital impressions regarding clarity, visual appeal, and overall usability. Their feedback helped highlight which visuals captured their attention first and whether the report effectively communicated the type of information an executive might expect.
- **Consulting with data analytics professionals:** <br>
I would like to review the report with an experienced data analyst or BI developer to receive guidance on improving the data model, visual design, DAX efficiency, and storytelling flow.
- **Integrating richer datasets:** <br>
Incorporating additional data such as product cost, marketing spend, or customer behavior would enable more comprehensive analysis. This would allow for profitability metrics, customer segmentation, and deeper insights into factors driving performance.
- **Enhancing interactivity and personalization:** <br>
Future versions could include dynamic navigation elements, drill-through pages, or tooltip pages to create a more immersive and intuitive user experience. These enhancements would allow users to move from high-level overviews to detailed diagnostics more efficiently, allowing for more in-depth analysis.